## Functions & methods 

All functions are non-data descriptors which return bound methods when they are invoked from an object


In [17]:
def hello_world():
    print("hello world")
    
print(hello_world.__get__)

<method-wrapper '__get__' of function object at 0x00000220EE034D90>


Function are object and this is roughly the implementation. 
```python
class Function(object):
    def __get__(self, obj, objtype=None):
        "Simulate func_descr_get() in Objects/funcobject.c"
        if obj is None:
            return self
        return types.MethodType(self, obj)
```

In [18]:
class A:
    def foo(self):
        print(self)
        
    print(foo.__get__)
print(A().foo)

<method-wrapper '__get__' of function object at 0x00000220EE034840>
<bound method A.foo of <__main__.A object at 0x00000220EE051630>>


This is last week's implementation of @staticmethod, back then we didn't understand why this was working. 

In [22]:
class static:
    def __init__(self, f):
        print(f"__init__ receive {f}")
        self.f = f
    
    def __get__(self, o, c):
        print(f"__get__ returns {self.f}")
        return self.f

class B:
    @static
    def foo(message):
        print(message)
        
print(f"B().foo : {B().foo}")
B.foo("hello")

__init__ receive <function B.foo at 0x00000220EE056488>
__get__ returns <function B.foo at 0x00000220EE056488>
B().foo : <function B.foo at 0x00000220EE056488>
__get__ returns <function B.foo at 0x00000220EE056488>
hello


__Same code than above but step by step.__

In [21]:
class static:
    def __init__(self, f):
        print(f"__init__ receive {f}")
        self.f = f
    
    def __get__(self, o, c):
        print(f"__get__ returns {self.f}")
        return self.f

def foo(message):
    print(message)

class B:
    pass

static_foo = static(foo)

B.foo = static_foo

print(f"B().foo : {B().foo}")
B.foo("hello")

__init__ receive <function foo at 0x00000220EE034B70>
__get__ returns <function foo at 0x00000220EE034B70>
B().foo : <function foo at 0x00000220EE034B70>
__get__ returns <function foo at 0x00000220EE034B70>
hello


What we actually do with static is that we wrap a descriptor in another descriptor. 

When we do `B().foo()`

- foo is accessed through an object so the if foo is a descriptor we cal its \__get__ method
- static_foo is a descriptior so static_foo.\__get__ is called
- static_foo.\__get__ return foo, here even if foo is a descriptor it's not accessed *directly* through an object so the descriptor protocol is not applied and it's called normaly. 
